# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages

import sqlite3
import pandas as pd

conn = sqlite3.Connection("data/data2.sqlite")

In [3]:
# Your code here; create the connection
qtest = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
"""

pd.read_sql(qtest, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [12]:
# Your code here
q1 = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM 
    customers
WHERE 
    customerNumber IN (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31')
;
"""

pd.read_sql(q1, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [20]:
# Your code here

q2 = """
SELECT 
    productName,
    (SELECT SUM(quantityOrdered) FROM orderdetails WHERE orderdetails.productCode = products.productCode) AS totalSold
FROM 
    products
ORDER BY 
    totalSold DESC;

"""

pd.read_sql(q2, conn)

,productName,totalSold
0,1992 Ferrari 360 Spider red,1808.0
1,1937 Lincoln Berline,1111.0
2,American Airlines: MD-11S,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,1076.0
4,1930 Buick Marquette Phaeton,1074.0
...,...,...
105,1911 Ford Town Car,832.0
106,1936 Mercedes Benz 500k Roadster,824.0
107,1970 Chevy Chevelle SS 454,803.0
108,1957 Ford Thunderbird,767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [28]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

q3 = """
SELECT 
    p.productName,
    COUNT(DISTINCT o.customerNumber) as num_customers
    
FROM 
    products p
    JOIN  orderdetails od ON p.productCode = od.productCode
    JOIN  orders o ON od.orderNumber = o.orderNumber
    
GROUP BY 
    p.productName    
    
ORDER BY 
    num_customers DESC;

"""
pd.read_sql(q3, conn)


,productName,num_customers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [43]:
q10 = """

SELECT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    of.city,
    of.officeCode
FROM
    employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    JOIN offices of ON e.officeCode = of.officeCode
GROUP BY
    e.employeeNumber
HAVING
    COUNT(DISTINCT o.customerNumber) < 20
ORDER BY
    e.employeeNumber ASC;



"""
pd.read_sql(q10, conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


In [41]:
# Your code here

qtest4 = """
SELECT 
    COUNT(DISTINCT o.customerNumber) AS num_customers,
    e.employeeNumber,
    e.firstName,
    e.lastName,
    of.city,
    of.officeCode
FROM 
    products p
    JOIN orderdetails od ON p.productCode = od.productCode
    JOIN orders o ON od.orderNumber = o.orderNumber
    JOIN customers c ON o.customerNumber = c.customerNumber
    JOIN employees e ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN offices of ON e.officeCode = of.officeCode
GROUP BY 
    e.employeeNumber,
    e.firstName,
    e.lastName,
    of.city,
    of.officeCode
HAVING 
    COUNT(DISTINCT o.customerNumber) < 20
ORDER BY 
    num_customers DESC;



"""
pd.read_sql(qtest4, conn)

,num_customers,employeeNumber,firstName,lastName,city,officeCode
0,10,1401,Pamela,Castillo,Paris,4
1,9,1504,Barry,Jones,London,7
2,8,1323,George,Vanauf,NYC,3
3,8,1501,Larry,Bott,London,7
4,7,1370,Gerard,Hernandez,Paris,4
5,6,1165,Leslie,Jennings,San Francisco,1
6,6,1166,Leslie,Thompson,San Francisco,1
7,6,1188,Julie,Firrelli,Boston,2
8,6,1216,Steve,Patterson,Boston,2
9,6,1286,Foon Yue,Tseng,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [36]:
# Your code here

qtest4 = """
SELECT 
    p.productName,
    COUNT(DISTINCT o.customerNumber) AS num_customers
FROM 
    products p
    JOIN orderdetails od ON p.productCode = od.productCode
    JOIN orders o ON od.orderNumber = o.orderNumber
GROUP BY 
    p.productName
HAVING 
    COUNT(DISTINCT o.customerNumber) < 20
ORDER BY 
    num_customers DESC;


"""
pd.read_sql(qtest4, conn)

,productName,num_customers
0,1958 Chevy Corvette Limited Edition,19
1,2002 Chevy Corvette,18
2,1969 Chevrolet Camaro Z28,18
3,1952 Citroen-15CV,18
4,1949 Jaguar XK 120,18


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!